In [1]:
# for rvl dataset 
dataloc = "data/images/"
outputloc = "output"
labels = "data/labels"

# for my small dataset 
#dataloc = 
#outputloc = 
#data/labels

In [2]:
import os
import threading
import time
import datetime, random
import pandas as pd
from tqdm.notebook import tqdm

from IPython.display import HTML
import plotly.express as px

In [4]:
img_files = []

train_df = pd.read_csv(os.path.join(labels, 'train.txt'), delimiter=' ', header= None)
train_df = train_df[(train_df[1]!=3)]
train_df.columns=['dir','cat']

test_df = pd.read_csv(os.path.join(labels, 'test.txt'), delimiter=' ', header= None)
test_df = test_df[(test_df[1]!=3)]
test_df.columns=['dir','cat']

val_df = pd.read_csv(os.path.join(labels, 'val.txt'), delimiter=' ', header= None)
val_df = val_df[(val_df[1]!=3)]
val_df.columns=['dir','cat']

In [22]:
print(val_df[val_df['dir'].str.contains('1555')])

                                                    dir  cat
1510       imagesb/b/u/u/buu91f00/00001554_00001555.tif   15
1657       imagesy/y/t/r/ytr10a00/10021551_10021555.tif    6
2399   imagesc/c/q/u/cqu29e00/2501555807_2501555811.tif    5
2840              imagesr/r/o/r/ror49e00/2501555248.tif    5
7073              imagesi/i/e/y/iey00f00/0000451555.tif   10
11045             imagesv/v/j/z/vjz08e00/1002415553.tif   12
12851               imagesz/z/l/h/zlh6aa00/11315554.tif   11
12870             imagesm/m/y/i/myi35c00/2069715553.tif    7
13960             imagesz/z/j/p/zjp61a00/2051555259.tif    7
17388             imagesg/g/u/a/gua48e00/1005155558.tif    6
17913  imagesm/m/t/g/mtg32e00/2501415554_2501415566.tif    5
17943             imagesn/n/v/n/nvn49e00/2501555586.tif    1
19542      imagesw/w/t/o/wto00e00/91555722_91555724.tif   11
19640             imagesi/i/b/o/ibo46e00/2024215558.tif    9
19853          imagesx/x/y/d/xyd00d00/50315558-5559.tif   14
23863             images

In [5]:
# now remove handwriting im not interested in this
train_df.head()

,dir,cat
0,imagesq/q/o/c/qoc54c00/80035521.tif,15
1,imagese/e/w/c/ewc23d00/513280028.tif,1
2,imagesw/w/b/t/wbt26e00/2053453161.tif,7
3,imagesm/m/k/m/mkm05e00/2040792992_2040792994.tif,10
5,imagesq/q/v/t/qvt06d00/50525666-5667.tif,14


In [18]:
total_train = 10000

def get_rep_sample(img_df, prefix = "train "):
    if prefix == "trian":
        sample_size = total_train
    else:
        sample_size = int(total_train * .1)
    
    return img_df.sample(n= sample_size)

In [19]:
#get_rep_sample_size(train_df)

In [20]:
#must be added to get kraken to run in cmd 
os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'

def image_ocr(imagefiles, prefix = "" ):
    location = []
    location.append(["algor",'time','cat','output_dir'])
    print("Total number of images to go though {} with 2 different alorithsm".format(imagefiles.shape[0]))
    for fileinput in tqdm(imagefiles.itertuples(), total = imagefiles.shape[0]):
        
        input_file = os.path.join(dataloc, fileinput.dir)
        output_file_name = fileinput.dir.split("/")[-1][:-4]
        output_file_kraken = "{}/{}/{}.txt".format(outputloc, "kraken", "{}_{}".format(prefix , output_file_name))
        output_file_tesseract = "{}/{}/{}.txt".format(outputloc, "tesseract", "{}_{}".format(prefix , output_file_name))
        
        #print(input_file)
        #print(output_file_kraken)
        
        pro_time = time.process_time()
        system_cmd = "kraken -i {} {} binarize segment ocr -m en_best.mlmodel".format(input_file, output_file_kraken)
        os.system(system_cmd)
        location.append(["kraken", time.process_time() - pro_time, fileinput.cat, output_file_kraken])

        system_cmd = "tesseract {} {} ".format(input_file, output_file_tesseract)
        pro_time = time.process_time()
        os.system(system_cmd)
        location.append(["tesseract", time.process_time() - pro_time, fileinput.cat, output_file_tesseract])
        
    return location

  

In [21]:
pd.DataFrame(image_ocr(get_rep_sample(train_df), prefix = "train")).to_csv("rvl_subset_train_locations.csv")

Total number of images to go though 1 with 2 different alorithsm


  0%|          | 0/1 [00:00<?, ?it/s]

In [49]:
pd.DataFrame(image_ocr(get_rep_sample(val_df, prefix = 'val'), prefix = "val")).to_csv("rvl_subset_val_locations.csv")

Total number of images to go though 100 with 2 different alorithsm


  0%|          | 0/100 [00:00<?, ?it/s]

Pandas(Index=8820, dir='imagesd/d/w/m/dwm87d00/2073354293.tif', cat=1)
Pandas(Index=16757, dir='imagesp/p/o/n/pon14d00/507806764.tif', cat=4)
Pandas(Index=37411, dir='imagesv/v/l/p/vlp72f00/tob09317.83.tif', cat=9)
Pandas(Index=17585, dir='imagesh/h/w/r/hwr62d00/83348495.tif', cat=1)
Pandas(Index=24567, dir='imagesp/p/g/p/pgp81a00/1000377236_1000377239.tif', cat=13)
Pandas(Index=11076, dir='imagesu/u/f/f/uff45a00/80317042.tif', cat=13)
Pandas(Index=35698, dir='imagesd/d/r/x/drx27d00/2073229203_9205.tif', cat=1)
Pandas(Index=19196, dir='imagesr/r/r/s/rrs23f00/0000061584.tif', cat=7)
Pandas(Index=37363, dir='imagesw/w/o/f/wof53e00/2050871667.tif', cat=1)
Pandas(Index=33031, dir='imagest/t/d/j/tdj29c00/2084029786a.tif', cat=2)
Pandas(Index=26129, dir='imagesq/q/v/j/qvj89c00/50267173-7186.tif', cat=14)
Pandas(Index=35313, dir='imagesi/i/y/y/iyy95d00/50483850-3851.tif', cat=14)
Pandas(Index=12582, dir='imagesp/p/l/k/plk93d00/508473757.tif', cat=7)
Pandas(Index=14584, dir='imagese/e/f/x/efx0

In [50]:
pd.DataFrame(image_ocr(get_rep_sample(test_df, prefix = 'test'), prefix = "test")).to_csv("rvl_subset_test_locations.csv")

Total number of images to go though 100 with 2 different alorithsm


  0%|          | 0/100 [00:00<?, ?it/s]

Pandas(Index=28386, dir='imagest/t/m/l/tml82d00/80302096_2097.tif', cat=4)
Pandas(Index=39004, dir='imagesf/f/f/q/ffq07e00/2050240673.tif', cat=6)
Pandas(Index=17365, dir='imagesd/d/j/w/djw29e00/2501382690.tif', cat=12)
Pandas(Index=13471, dir='imagesl/l/r/f/lrf02d00/71508305.tif', cat=4)
Pandas(Index=3750, dir='imagesj/j/n/l/jnl23a00/501077300+-7303.tif', cat=4)
Pandas(Index=26782, dir='imagesi/i/j/u/iju10e00/94395729.tif', cat=8)
Pandas(Index=25637, dir='imagesy/y/u/a/yua36d00/50728441-8442.tif', cat=14)
Pandas(Index=16042, dir='imagesw/w/f/o/wfo62d00/86066705_6706.tif', cat=10)
Pandas(Index=14402, dir='imagesq/q/w/a/qwa63e00/2030157619.tif', cat=8)
Pandas(Index=21017, dir='imagesq/q/q/k/qqk91c00/2078771290a.tif', cat=2)
Pandas(Index=31337, dir='imagesl/l/s/d/lsd65e00/2045342889_2045342917.tif', cat=10)
Pandas(Index=22228, dir='imagesd/d/s/t/dst08e00/1002394467_1002394469.tif', cat=13)
Pandas(Index=10962, dir='imagesq/q/z/s/qzs62e00/2040214440_2040214442.tif', cat=13)
Pandas(Index=60

In [ ]:
tr